<a href="https://colab.research.google.com/github/staerkjoe/NLP_colab/blob/main/AML4NLP2025_E04_Intro_W%26B_Weave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction: Eploring Weights & Biases Weave

ITU KSAMLDS1KU - Advanced Machine Learning for Data Science 2025

by Jonathan Tiedchen, Eisuke Okuda, Stefan Heinrich,
& material by Bertram Højer, Kevin Murphy, and Chris Bishop.

All info and static material: https://learnit.itu.dk/course/view.php?id=3024752

-------------------------------------------------------------------------------

### 1. Install dependencies

In [1]:
!pip install wandb weave transformers datasets torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 6.7 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=dfc68b75968b06b37675c0f2a433530c8d8729ab96d9df62c1f1c27e29e787f6
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa

### 2. Imports

In [1]:
import wandb
import weave
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

### 3. Initialize W&B and Weave

In [13]:
# Login to W&B
wandb.login()

# Start a new run
wandb_run = wandb.init(
    project="llm_prompt_logging",
    config={
        "model": "gpt2",
        "max_new_tokens": 20,
        "temperature": 0.2
    }
)

# Initialize Weave client
client = weave.init("jojs-it-universitetet-i-k-benhavn/llm_prompt_logging")

wandb: Initializing weave.


Output()

weave: wandb version 0.22.0 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: jojs.
weave: View Weave data at https://wandb.ai/jojs-it-universitetet-i-k-benhavn/llm_prompt_logging/weave


Output()

weave: wandb version 0.22.0 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: jojs.
weave: View Weave data at https://wandb.ai/jojs-it-universitetet-i-k-benhavn/llm_prompt_logging/weave


### 4. Load a Small Model (tiny GPT-2)

In [14]:
model_name = wandb.config["model"]

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### 5. Define a Prompt Function with Logging

In [15]:
def prompt_model(question, temperature=0.7, max_new_tokens=50):
    # Create Weave call
    call = client.create_call(
        op="prompt_model",
        inputs={
            "question": question,
            "temperature": temperature,
            "max_new_tokens": max_new_tokens
        }
    )

    # Encode input
    input_text = f"Q: {question}\nA:"
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = output_text.split("A:")[-1].strip()

    # Log to W&B
    wandb.log({"question": question, "answer": answer})

    # Finish Weave call
    client.finish_call(call, output={"answer": answer})

    return answer


### 6. Run a few examples

In [16]:
questions = [
    "What is 2+2?",
    "Explain the capital of France.",
    "Who wrote the book 1984?"
]

for q in questions:
    ans = prompt_model(q, temperature=wandb.config["temperature"])
    print(f"Q: {q}\nA: {ans}\n")


weave: 🍩 https://wandb.ai/jojs-it-universitetet-i-k-benhavn/llm_prompt_logging/r/call/01996c18-7260-76d1-b638-46c75eb325b1
weave: 🍩 https://wandb.ai/jojs-it-universitetet-i-k-benhavn/llm_prompt_logging/r/call/01996c18-7e3c-7892-94f3-9261141a5850


Q: What is 2+2?
A: 2+2 is a number that is used to represent the number of times a given number is equal to



weave: 🍩 https://wandb.ai/jojs-it-universitetet-i-k-benhavn/llm_prompt_logging/r/call/01996c18-89c4-7914-9ce6-ca115c081c51


Q: Explain the capital of France.
A: The capital of France is Paris. It is the capital of France. It is the capital of France. It is the capital of France. It is the capital of France. It is the capital of France. It is the capital of France. It

Q: Who wrote the book 1984?
A: I think it was a very good book. I think it was written by a very good writer. I think it was written by a very good writer. I think it was written by a very good writer. I think it was written by a very



### 7. Finish Run

In [17]:
wandb.finish()


answer,I think it was a ver...
question,Who wrote the book 1...
